In [ ]:
path='/content/drive/MyDrive/VKR/'
path_in=path+'input/'
path_out=path+'output/'
path_tmp='/content/'

In [ ]:
!pip install -U pyflagser
!pip install pympler

In [ ]:
import pandas as pd
import networkx as nx
import numpy as np
import scipy, scipy.stats
import matplotlib.pyplot as plt
import os
import shutil
from collections import Counter
import seaborn as sns
from sklearn.linear_model import LinearRegression
import pyflagser
import json
from sklearn.cluster import KMeans
from pympler import asizeof

In [ ]:
import matplotlib
matplotlib.use('Agg')

## Read graph

In [ ]:
G = nx.read_graphml(path_out+"rus_main.graphml")
name='rus/'
G.number_of_edges(), G.number_of_nodes()

In [ ]:
G = nx.read_graphml(path_out+"en_r1_main.graphml")
name='en_r1/'
G.number_of_edges(), G.number_of_nodes()

In [ ]:
G = nx.read_graphml(path_out+"en_r123_main.graphml")
name='en_r123/'
G.number_of_edges(), G.number_of_nodes()

In [ ]:
G = nx.read_graphml(path_out+"dutch_main.graphml")
name='dutch/'
G.number_of_edges(), G.number_of_nodes()

In [ ]:
G = nx.read_graphml(path_out+"USF_main.graphml")
name='USF/'
G.number_of_edges(), G.number_of_nodes()

## defs

In [ ]:
def mle_power_law_params(degree_sequence): #find params of power law distribution of degree
    x_min = max(np.floor(np.min(degree_sequence)), 1)
    alpha = alpha_lin_bins(degree_sequence[degree_sequence>=x_min], 1000)
    D, _ = scipy.stats.kstest(rvs=degree_sequence[degree_sequence>=x_min], cdf=power_law_cdf, args = (alpha, x_min))
    for i in range(int(max(np.floor(np.min(degree_sequence)), 1))+1, int(np.floor(np.max(degree_sequence)))+1):
        alpha_i = alpha_lin_bins(degree_sequence[degree_sequence>=i], 1000)
        D_i, _ = scipy.stats.kstest(rvs=degree_sequence[degree_sequence>=i], cdf=power_law_cdf, args = (alpha_i, i))
        if D_i < D:
            D = D_i
            x_min = i
            alpha = alpha_i
    return (alpha, x_min)

def power_law_cdf(x, alpha=3.5, x_min=1):
    f = np.maximum(1 - x ** (-alpha+1) / x_min ** (1 - alpha), 0)
    return f

def power_law_pdf(x, alpha=3.5, x_min=1):
    C = (alpha - 1) / x_min ** (1 - alpha)
    return C * x ** (-alpha)

def alpha_lin_bins(x_train, bins):
    hist, bin_edges = np.histogram(x_train, bins=bins, density=True)
    bin_centers = (bin_edges[1:] + bin_edges[:-1]) / 2
    log_x=np.log(bin_centers[hist > 0]).reshape(-1, 1)
    e_cdf=np.log(hist[hist > 0]).reshape(-1, 1)
    lin_reg = LinearRegression(fit_intercept=False)
    lin_reg.fit(log_x, e_cdf)
    return -lin_reg.coef_[0][0]

def labels_list_parameters1(graph, layout, ct, scale=10, maxc=0, name=False, cut=True): #function for making labels size dependent on centrality
    list_of_params = []
    vals=ct.values() if isinstance(ct, dict) else ct
    for c in set(vals):
        params = {}
        params['G'] = graph
        params['pos'] = layout
        params['labels'] = {}
        params['font_size'] = c*scale+16 if c>=maxc else 6
        for node in graph.nodes:
            # print(ct[node])
            if ct[node] == c:
                params['labels'][node] = graph.nodes()[node]['name'] if name else str(node)
                if c<maxc and cut:
                    params['labels'][node] =' '
        list_of_params.append(params)
    return list_of_params

def calculate_graph_features(G, save_to, csv=False, num=0, diam=True): #common graph features: see prints and titles for info
  print("Number of nodes:", G.number_of_nodes())
  print("Number of edges:", G.number_of_edges())
  if nx.is_strongly_connected(G) and diam:
    d=nx.diameter(G)
    r=nx.radius(G)
    a=round(nx.average_shortest_path_length(G), 4)
    print("Diameter:", d)
    print("Radius:", r)
    print("ASPL:", round(a))
  elif diam:
    print("Graph not strongly connected, diam et al for max SCC")
    sub=G.subgraph(max(nx.strongly_connected_components(G), key=len))
    d=nx.diameter(sub)
    r=nx.radius(sub)
    a=round(nx.average_shortest_path_length(sub), 4)
    print("Diameter:", d)
    print("Radius:", r)
    print("ASPL:", round(a))
  else:
    print("Graph not strongly connected and parameter diam=False, skipping diam, etc.")
    d=np.inf
    r=np.inf
    a=np.inf
  print('Transitivity:', round(nx.transitivity(G), 4))
  print('ACC:', round(nx.average_clustering(G), 4))
  print('Clique number:', nx.graph_clique_number(G.to_undirected()))
  print('Density:', round(nx.density(G), 4))
  print('Average node degree: {0:.2f}'.format(np.array(list(dict(G.degree()).values())).mean()))
  print('Node degree variance: {0:.2f}'.format(np.array(list(dict(G.degree()).values())).var()))
  print('Average node in-degree: {0:.2f}'.format(np.array(list(dict(G.in_degree()).values())).mean()))
  print('Node in-degree variance: {0:.2f}'.format(np.array(list(dict(G.in_degree()).values())).var()))
  print('Average node out-degree: {0:.2f}'.format(np.array(list(dict(G.out_degree()).values())).mean()))
  print('Node out-degree variance: {0:.2f}'.format(np.array(list(dict(G.out_degree()).values())).var()))

  if isinstance(csv, pd.DataFrame):
    csv.loc[num]=[G.number_of_nodes(), G.number_of_edges(), d, r, a, 
                  round(nx.transitivity(G), 4), round(nx.average_clustering(G), 4), nx.graph_clique_number(G.to_undirected()),
                  round(nx.density(G), 4), np.array(list(dict(G.degree()).values())).mean(), 
                  np.array(list(dict(G.degree()).values())).var(), np.array(list(dict(G.in_degree()).values())).mean(),
                  np.array(list(dict(G.in_degree()).values())).var(), np.array(list(dict(G.out_degree()).values())).mean(),
                  np.array(list(dict(G.out_degree()).values())).var(), 0, 0, 0, 0, 0, 0, "", "", "", "","", "","", "","", ""]

  d=(np.array(nx.degree_histogram(G))/G.number_of_nodes())
  plt.rcParams['font.size'] = '14'
  plt.figure(figsize=(8, 6), dpi=200)
  plt.title("Degree distribution")
  plt.xlabel("Node degree")
  plt.ylabel("Fraction of degree")
  plt.bar(range(len(d)), d);
  plt.savefig(save_to+'degree_gen.png')
  plt.close('All')

  ind=np.array(list(dict(G.in_degree()).values()))
  h=Counter(ind)
  h=dict(h)
  for i in range(max(ind)):
    if not i in h:
      h[i]=0
  h=dict(sorted(h.items()))
  plt.rcParams['font.size'] = '14'
  plt.figure(figsize=(8, 6), dpi=200)
  plt.title("In-degree distribution")
  plt.xlabel("Node in-degree")
  plt.ylabel("Fraction of degree")
  plt.bar(list(h.keys())[:1000], np.array(list(h.values())[:1000])/G.number_of_nodes());
  plt.savefig(save_to+'degree_in.png')
  plt.close('All')

  outd=np.array(list(dict(G.out_degree()).values()))
  h=Counter(outd)
  h=dict(h)
  for i in range(max(outd)):
    if not i in h:
      h[i]=0
  h=dict(sorted(h.items()))
  plt.rcParams['font.size'] = '14'
  plt.figure(figsize=(8, 6), dpi=200)
  plt.title("Out-degree distribution")
  plt.xlabel("Node out-degree")
  plt.ylabel("Fraction of degree")
  plt.bar(list(h.keys())[:1000], np.array(list(h.values())[:1000])/G.number_of_nodes());
  plt.savefig(save_to+'degree_out.png')
  plt.close('All')

  

  hist, bin_edges = np.histogram(ind, bins=1000, density=True)
  bin_centers = (bin_edges[1:] + bin_edges[:-1]) / 2
  plt.rcParams['font.size'] = '14'
  plt.figure(figsize=(8, 6), dpi=200)
  plt.scatter(bin_centers[hist > 0], hist[hist > 0], s=5, label="Real PDF")
  plt.title('In-degree distribution')
  hat_alpha, hat_x_min = mle_power_law_params(ind)
  x_space = np.linspace(hat_x_min, ind.max(), 100)
  plt.plot(x_space, power_law_pdf(x_space, hat_alpha, hat_x_min), 
          label='Estimated PDF', c='tab:orange')
  plt.xscale('log')
  plt.yscale('log')
  plt.legend();
  plt.savefig(save_to+'degree_in_estim.png')
  plt.close('All')
  print('Power law params: gamma=', round(hat_alpha, 4), 'k_min=', hat_x_min)
  if isinstance(csv, pd.DataFrame):
    csv.loc[num, 'gamma']=round(hat_alpha, 4)
    csv.loc[num, 'k_min']=hat_x_min

  plt.rcParams['font.size'] = '14'
  plt.figure(figsize=(8, 6), dpi=200) #plotting distribution of local clustering coefficients
  plt.title("Local clustering coefficient distribution")
  plt.xlabel("Clustering coefficient")
  plt.ylabel("Number of nodes")
  plt.hist(list(nx.clustering(G).values()), bins=20) #dispersed in 20 bins automatically
  plt.savefig(save_to+'LCC.png')
  plt.close('All')

  s=list(set(list(dict(G.in_degree).values())))
  maps={s[i]: i for i in range(len(s))}
  mix=nx.degree_mixing_matrix(G, x='in', y='in', mapping=maps)
  deg={(i, j): mix[i, j] for i in range(mix.shape[0]) for j in range(i, mix.shape[1])}
  print("Assortativity coefficient (in-in):", round(nx.degree_pearson_correlation_coefficient(G, x='in', y='in'), 4))
  if isinstance(csv, pd.DataFrame):
    csv.loc[num, 'AII']=round(nx.degree_pearson_correlation_coefficient(G, x='in', y='in'), 4)

  # for i, j in sorted(deg, key=deg.get, reverse=True)[:20]:
  #     k=deg[(i,j)]
  #     if k>0:
  #         print(s[i], s[j], k)
  plt.figure(figsize=(12, 8),dpi=200)
  k=max(1,len(maps)//10)
  r=sns.heatmap(
          mix[-1::-1])
  r.set_xticks(list(maps.values())[::k], labels=list(maps.values())[::k])
  r.set_yticks(list(maps.values())[::k], labels=list(maps.values())[::k][-1::-1])
  plt.title("Degree correlation matrix (in-degree vs in-degree)")
  plt.savefig(save_to+'DCM_in_in.png')
  plt.close('All')

  x=np.array(sorted(set(dict(G.in_degree()).values()))).reshape(-1,1)
  z=nx.average_degree_connectivity(G, source='in', target='in')
  y=np.array([z[i[0]] for i in x]).reshape(-1,1)
  lin_reg = LinearRegression()
  lin_reg.fit(x, y)
  pr=lin_reg.predict(x)
  plt.rcParams['font.size'] = '14'
  plt.figure(figsize=(8,6), dpi=200)
  plt.scatter(x,y, label='func')
  plt.plot(x,pr, color='orange', label='fit')
  plt.xlabel('degree')
  plt.ylabel('DCF')
  plt.title('DCF (in-degree vs in-degree)')
  plt.legend()
  plt.savefig(save_to+'DCF_in_in.png')
  plt.close('All')

  s=list(set(list(dict(G.out_degree).values())))
  maps={s[i]: i for i in range(len(s))}
  mix=nx.degree_mixing_matrix(G, x='out', y='out', mapping=maps)
  deg={(i, j): mix[i, j] for i in range(mix.shape[0]) for j in range(i, mix.shape[1])} #same top as for cities
  print("Assortativity coefficient (out-out):", round(nx.degree_pearson_correlation_coefficient(G, x='out', y='out'), 4))
  if isinstance(csv, pd.DataFrame):
    csv.loc[num, 'AOO']=round(nx.degree_pearson_correlation_coefficient(G, x='out', y='out'), 4)
  # for i, j in sorted(deg, key=deg.get, reverse=True)[:20]:
  #     k=deg[(i,j)]
  #     if k>0:
  #         print(s[i], s[j], k)
  plt.figure(figsize=(12, 8),dpi=200)
  k=max(1,len(maps)//10)
  r=sns.heatmap(
          mix[-1::-1])
  r.set_xticks(list(maps.values())[::k], labels=list(maps.values())[::k])
  r.set_yticks(list(maps.values())[::k], labels=list(maps.values())[::k][-1::-1])
  plt.title("Degree correlation matrix (out-degree vs out-degree)")
  plt.savefig(save_to+'DCM_out_out.png')
  plt.close('All')

  x=np.array(sorted(set(dict(G.out_degree()).values()))).reshape(-1,1)
  z=nx.average_degree_connectivity(G, source='out', target='out')
  y=np.array([z[i[0]] for i in x]).reshape(-1,1)
  lin_reg = LinearRegression()
  lin_reg.fit(x, y)
  pr=lin_reg.predict(x)
  plt.rcParams['font.size'] = '14'
  plt.figure(figsize=(8,6), dpi=200)
  plt.scatter(x,y, label='func')
  plt.plot(x,pr, color='orange', label='fit')
  plt.xlabel('degree')
  plt.ylabel('DCF')
  plt.title('DCF (out-degree vs out-degree)')
  plt.legend()
  plt.savefig(save_to+'DCF_out_out.png')
  plt.close('All')

  s=list(set(list(dict(G.in_degree).values())))
  maps={s[i]: i for i in range(len(s))}
  mix=nx.degree_mixing_matrix(G, x='in', y='out', mapping=maps)
  deg={(i, j): mix[i, j] for i in range(mix.shape[0]) for j in range(i, mix.shape[1])} #same top as for cities
  print("Assortativity coefficient (in-out):", round(nx.degree_pearson_correlation_coefficient(G, x='in', y='out'), 4))
  if isinstance(csv, pd.DataFrame):
    csv.loc[num, 'AIO']=round(nx.degree_pearson_correlation_coefficient(G, x='in', y='out'), 4)
  # for i, j in sorted(deg, key=deg.get, reverse=True)[:20]:
  #     k=deg[(i,j)]
  #     if k>0:
  #         print(s[i], s[j], k)
  plt.figure(figsize=(12, 8),dpi=200)
  k=max(1,len(maps)//10)
  r=sns.heatmap(
          mix[-1::-1])
  r.set_xticks(list(maps.values())[::k], labels=list(maps.values())[::k])
  r.set_yticks(list(maps.values())[::k], labels=list(maps.values())[::k][-1::-1])
  plt.title("Degree correlation matrix (in-degree vs out-degree)")
  plt.savefig(save_to+'DCM_in_out.png')
  plt.close('All')

  x=np.array(sorted(set(dict(G.in_degree()).values()))).reshape(-1,1)
  z=nx.average_degree_connectivity(G, source='in', target='out')
  y=np.array([z[i[0]] for i in x]).reshape(-1,1)
  lin_reg = LinearRegression()
  lin_reg.fit(x, y)
  pr=lin_reg.predict(x)
  plt.rcParams['font.size'] = '14'
  plt.figure(figsize=(8,6), dpi=200)
  plt.scatter(x,y, label='func')
  plt.plot(x,pr, color='orange', label='fit')
  plt.xlabel('degree')
  plt.ylabel('DCF')
  plt.title('DCF (in-degree vs out-degree)')
  plt.legend()
  plt.savefig(save_to+'DCF_in_out.png')
  plt.close('All')

  s=list(set(list(dict(G.out_degree).values())))
  maps={s[i]: i for i in range(len(s))}
  mix=nx.degree_mixing_matrix(G, x='out', y='in', mapping=maps)
  deg={(i, j): mix[i, j] for i in range(mix.shape[0]) for j in range(i, mix.shape[1])} #same top as for cities
  print("Assortativity coefficient (out-in):", round(nx.degree_pearson_correlation_coefficient(G, x='out', y='in'), 4))
  if isinstance(csv, pd.DataFrame):
    csv.loc[num, 'AOI']=round(nx.degree_pearson_correlation_coefficient(G, x='out', y='in'), 4)
  # for i, j in sorted(deg, key=deg.get, reverse=True)[:20]:
  #     k=deg[(i,j)]
  #     if k>0:
  #         print(s[i], s[j], k)
  plt.figure(figsize=(12, 8),dpi=200)
  k=max(1,len(maps)//10)
  r=sns.heatmap(
          mix[-1::-1])
  r.set_xticks(list(maps.values())[::k], labels=list(maps.values())[::k])
  r.set_yticks(list(maps.values())[::k], labels=list(maps.values())[::k][-1::-1])
  plt.title("Degree correlation matrix (out-degree vs in-degree)")
  plt.savefig(save_to+'DCM_out_in.png')
  plt.close('All')

  x=np.array(sorted(set(dict(G.out_degree()).values()))).reshape(-1,1)
  z=nx.average_degree_connectivity(G, source='out', target='in')
  y=np.array([z[i[0]] for i in x]).reshape(-1,1)
  lin_reg = LinearRegression()
  lin_reg.fit(x, y)
  pr=lin_reg.predict(x)
  plt.rcParams['font.size'] = '14'
  plt.figure(figsize=(8, 6), dpi=200)
  plt.scatter(x,y, label='func')
  plt.plot(x,pr, color='orange', label='fit')
  plt.xlabel('degree')
  plt.ylabel('DCF')
  plt.title('DCF (out-degree vs in-degree)')
  plt.legend()
  plt.savefig(save_to+'DCF_out_in.png')
  plt.close('All')
  ec=nx.eigenvector_centrality(G, max_iter=10000) #plotting graph - for clusters - comment if not needed
  for m, j in enumerate(sorted(ec, key=ec.get, reverse=True)):
    if m<10:
      print(j, ' -- ', round(ec[j], 4), end=', ')
      last=ec[j]
      if isinstance(csv, pd.DataFrame): 
        csv.loc[num, m]=j
  if len(G)<500:
    plt.figure(figsize=(30, 12), dpi=600)
    pos = nx.spring_layout(G)
    nx.draw_networkx_nodes(G, pos, node_size=[min(i*20*3.7**(i*15)+20, 0.35*20*3.7**(0.35*15)+20) for i in ec.values()], alpha=0.7,  node_color=[i for i in ec.values()], cmap=plt.cm.Oranges)
    nx.draw_networkx_edges(G, pos, alpha=0.08)
    for params in labels_list_parameters1(G, pos, ec, 25, last):
        nx.draw_networkx_labels(**params)
    plt.savefig(save_to+'layout.jpeg')
    plt.close('all')
  return csv

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def NB(G, save_to='', A=False, vals=False, vals1=False, dir=False, num=100, LM=False):
#calculate NB matrix, its num largest and smallest real part eigenvalues, plot them with circles
  shutil.copytree(path_in+'el_lib', 'el_lib')
  from el_lib import Alpha
  if dir:
    adj_sym=nx.to_numpy_array(G, weight=None)
    graph_sym = nx.from_numpy_array(adj_sym, create_using=nx.DiGraph)
    all_edg_sym = np.array(graph_sym.edges())
  else:
    pipeline_object = Alpha(G)
    adj = pipeline_object.adjacency_mat()
    _, adj_sym = pipeline_object.preprocessing_matrix(adj)
    all_edg_sym = pipeline_object.edges_extracting(adj_sym)
  if vals:
    vals_flow=np.load(vals)
    if vals1:
      vals_flow1=np.load(vals1)
      vals_flow=np.hstack([vals_flow, vals_flow1])
  else: 
    if A:
      A=scipy.sparse.load_npz(A)
    else:
      nds={int(i):[] for i in G.nodes()}
      ed={i: [] for i in range(all_edg_sym.shape[0])}
      for i in range(all_edg_sym.shape[0]):
        k, l = all_edg_sym[i]
        nds[k].append(i)
      for i in nds:
          if not i%500:
              print(i)
          for j in nds[i]:
              k, l = all_edg_sym[j]
              for m in nds[l]:
                q, r = all_edg_sym[m]
                if not r==k:
                  ed[j].append(m)
      degrees = np.sum(adj_sym, axis=1)
      B=np.zeros((1000, np.shape(all_edg_sym)[0]))
      s=0000
      m=0000
      file='NB_A.npz'
      for i in range(s, np.shape(all_edg_sym)[0]):
          for j in ed[i]:
            if degrees[all_edg_sym[j][0]] > 1:
              B[i%1000][j] = 1/(degrees[all_edg_sym[j][0]] - 1)
            else: 
              B[i%1000][j] = 1
          if not (i+1)%1000 and i>0:
            if not (i+1)%10000:
              print(i/np.shape(all_edg_sym)[0]*100, end='% ')
            if i==m+999:
              A=scipy.sparse.csr_matrix(B)
            else:
              A=scipy.sparse.vstack([A, scipy.sparse.csr_matrix(B)])
            B=np.zeros((1000, np.shape(all_edg_sym)[0]))
            if (i+1)%50000==0:
              scipy.sparse.save_npz(save_to+file, A)
              print('save')
              print(asizeof.asizeof(A)/1024/1024, end=' ')
              print(A.shape, A.nnz)
      if i<999:
        A=scipy.sparse.csr_matrix(B)
      else:
        A=scipy.sparse.vstack([A, scipy.sparse.csr_matrix(B)])
      A=A[:np.shape(all_edg_sym)[0]-m]
      scipy.sparse.save_npz(save_to+file, A)
      print(A.shape, A.nnz)
    if LM:
      vals_flow=scipy.sparse.linalg.eigs(A, min(A.shape[0]-2, num), which='LM', return_eigenvectors=False)
      np.save(save_to+'val_flow.npy', vals_flow)
      print('0')
    else:
      vals_flow=scipy.sparse.linalg.eigs(A, min(A.shape[0]-2, num), which='LR', return_eigenvectors=False)
      np.save(save_to+'val_flow.npy', vals_flow)
      print('1')
      if A.shape[0]>num:
        vals_flow1=scipy.sparse.linalg.eigs(A, min(A.shape[0]-2, num), which='SR',  return_eigenvectors=False)
        np.save(save_to+'/val_flow1.npy', vals_flow1)
        print('2')
        vals_flow=np.hstack([vals_flow, vals_flow1])
  degrees = np.sum(adj_sym, axis=1)
  cr_rad = np.sqrt(np.mean(np.array(degrees)/(np.array([x-1 if x>1 else max(x, 1) for x in degrees])))/(np.mean(degrees)))
  vals_sorted = vals_flow[vals_flow > cr_rad]
  order = np.argsort(-np.abs(np.array(vals_sorted)))
  max_clust_num=1000
  vals_sorted = np.array(vals_sorted)[order[1:max_clust_num]] 
  tail = np.shape(vals_sorted)[0]
  cr_rad, tail
  ld=(adj_sym.sum(axis=1)).max()
  avd=adj_sym.sum(axis=1).mean()
  r1=1/np.sqrt(max(avd-1, 1e-20))
  r2=1/(ld-1)
  r3=1
  plt.figure(figsize = (10, 10), dpi=200)
  plt.scatter(vals_flow.real, vals_flow.imag)
  plt.scatter(vals_sorted.real, vals_sorted.imag)
  circle1 = plt.Circle((0, 0), cr_rad, color='r', fill=False, linestyle='--', alpha=0.5, label='critical radius')
  plt.gca().add_patch(circle1)
  plt.title("Non-backtracking Laplacian spectrum")
  plt.legend()
  plt.savefig(save_to+'/vals.jpeg')
  vals_flow=1-vals_flow
  plt.figure(figsize = (10, 10), dpi=200)
  plt.scatter(vals_flow.real, vals_flow.imag)
  circle1 = plt.Circle((1, 0), r1, color='r', fill=False, linestyle='--', alpha=0.5, label=r'$\frac{1}{\sqrt{\alpha-1}}$')
  plt.gca().add_patch(circle1)
  circle2 = plt.Circle((1, 0), r2, color='g', fill=False, linestyle='--', alpha=0.5, label=r'$\frac{1}{\Delta-1}$')
  plt.gca().add_patch(circle2)
  circle3 = plt.Circle((1, 0), r3, color='y', fill=False, linestyle='--', alpha=0.5, label='1')
  plt.gca().add_patch(circle3)
  plt.title("Non-backtracking Laplacian spectrum")
  plt.legend()
  plt.savefig(save_to+'/vals2.jpeg')

In [ ]:
def filter(G, p=0.1): #filter edges by weight
  m=min(int(G.number_of_edges()*p), G.number_of_edges()-1)
  A=nx.to_numpy_array(G)
  w=sorted(nx.get_edge_attributes(G, 'weight').values(), reverse=True)[m]
  A[A<w]=0
  return nx.DiGraph(A)

In [ ]:
def calc_triangles(G, p_start=0, weighted=True): #calculate distribution of triangles
  stat=np.zeros((4, len(np.arange(p_start, 1.01, 0.1))))
  for num, p in enumerate(np.arange(p_start, 1.01, 0.1)):
    if weighted:
      sub=filter(G, p)
    else:
      sub=G
    t_type=np.zeros(4)
    ne=0
    sub2=nx.to_undirected(sub)
    for (i, j) in sub.edges():
      t=np.zeros(4)
      neig=nx.common_neighbors(sub2, i, j)
      for k in neig:
        if (j, k) in sub.edges() and (k, i) in sub.edges():
          t[0]+=1
        elif (k, j) in sub.edges() and (k, i) in sub.edges():
          t[1]+=1
        elif (j, k) in sub.edges() and (i, k) in sub.edges():
          t[2]+=1
        elif (i, k) in sub.edges() and (k, j) in sub.edges():
          t[3]+=1
      t=t/t.sum() if t.sum()>0 else t
      if t.sum()>0:
        ne+=1
        t_type+=t
    stat[:, num]=t_type/ne if ne>0 else t_type
  return stat

In [ ]:
def gen(n, m1=1, p=0.1): #model GEN1
  G=nx.erdos_renyi_graph(20, 0.1, directed=True)
  for i in range(20, n):
    if not i%(n//10):
      print(i)
    probs={}
    for j, k in G.edges():
      probs[(j, k)]=G.degree[j]+G.degree[k]
    m=list(probs.keys())
    p=np.array(list(probs.values()))
    p=p/p.sum()
    t=np.random.choice([i for i in range(len(m))], 3*m1, False, p) #add triangles type 2-4
    for num, j in enumerate(t):
      k, l = m[j]
      if num%3==0:
        G.add_edge(i, k)
        G.add_edge(i, l)
      elif num%3==1:
        G.add_edge(k, i)
        G.add_edge(l, i)
      elif num%3==2:
        G.add_edge(k, i)
        G.add_edge(i, l)
    j=0
    while not j==6*m1: #random edges
      k, l = np.random.choice(list(G.nodes()), 2, False)
      if not (k, l) in G.edges():
        G.add_edge(k, l)
        j+=1
  return G

In [ ]:
def gen2(n, tri, m1=1, p=0.1): #GEN2
  G=nx.erdos_renyi_graph(20, p, directed=True)
  for i in range(20, n):
    if not i%(n//10):
      print(i)
    probs={}
    for j, k in G.edges():
      probs[(j, k)]=G.degree[j]+G.degree[k]
    m=list(probs.keys())
    p=np.array(list(probs.values()))
    p=p/p.sum()
    t=np.random.choice([i for i in range(len(m))], 4*m1, False, p)

    for j in t:
      num=np.random.choice([i for i in range(4)], 1, p=tri)[0] #triangles any type
      k, l = m[j]
      if num==0:
        G.add_edge(l, i)
        G.add_edge(i, k)
      elif num==1:
        G.add_edge(i, k)
        G.add_edge(i, l)
      elif num==2:
        G.add_edge(k, i)
        G.add_edge(l, i)
      elif num==3:
        G.add_edge(k, i)
        G.add_edge(i, l)
    j=0
    while not j==4*m1:
      k, l = np.random.choice(list(G.nodes()), 2, False)
      if not (k, l) in G.edges():
        G.add_edge(k, l)
        j+=1
  return G

In [ ]:
def gen3(n, tri, load=False, file='net.graphml', m1=1, p=0.1): #GEN3
  if load:
    G=nx.read_graphml(file)
  else:
    G=nx.erdos_renyi_graph(20, p, directed=True)
  tri_use=tri.copy()
  for i in range(len(G), n):
    if load:
      i=str(i)
    probs={}
    for j, k in G.edges():
      probs[(j, k)]=G.degree[j]+G.degree[k]
    m=list(probs.keys())
    p=np.array(list(probs.values()))
    p=p/p.sum()
    t=np.random.choice([i for i in range(len(m))], 4*m1, False, p)

    for j in t:
      num=np.random.choice([i for i in range(4)], 1, p=tri_use)[0]
      k, l = m[j]
      if num==0:
        G.add_edge(l, i)
        G.add_edge(i, k)
      elif num==1:
        G.add_edge(i, k)
        G.add_edge(i, l)
      elif num==2:
        G.add_edge(k, i)
        G.add_edge(l, i)
      elif num==3:
        G.add_edge(k, i)
        G.add_edge(i, l)
    j=0
    while not j==4*m1:
      k, l = np.random.choice(list(G.nodes()), 2, False)
      if not (k, l) in G.edges():
        G.add_edge(k, l)
        j+=1
    if not int(i)%(n//100):
      tri_upd=calc_triangles(G, 1, False)[:,0] #update triangles distribution
      tri_use=np.array([max(0, i) for i in 2*tri-tri_upd])
      tri_use=tri_use/tri_use.sum()
    if not int(i)%(n//10):
      nx.write_graphml(G, file)
      print(i)
  return G

In [ ]:
def DL(G, to, num=300): #find normalised directed laplacian eigs
  A=nx.to_scipy_sparse_array(G)
  B=(A.T/A.sum(1)).T
  e, u = scipy.sparse.linalg.eigs(B, k=1, which='LR')
  S = np.abs(u).squeeze()
  S=S/S.sum()
  F=scipy.sparse.diags(S)
  F1=F.power(1/2)
  F2=F.power(-1/2)
  L=(F1@B@F2+F2@B.T@F1)/2
  L=scipy.sparse.csr_matrix(L)
  print('L')
  e, u=scipy.sparse.linalg.eigs(L, num, which='LR')
  plt.figure(figsize = (10, 10), dpi=200)
  plt.scatter(e.real, e.imag)
  # plt.axis('off')
  plt.show()
  plt.savefig(path_out+name+to+'vals1.jpeg')
  print('fin')
  return L

def DL_2(G, Gr, to, num=300): #find NDL eigs and compare to real (for models)
  A=nx.to_scipy_sparse_array(G)
  B=(A.T/A.sum(1)).T
  e, u = scipy.sparse.linalg.eigs(B, k=1, which='LR')
  S = np.abs(u).squeeze()
  S=S/S.sum()
  F=scipy.sparse.diags(S)
  F1=F.power(1/2)
  F2=F.power(-1/2)
  L=(F1@B@F2+F2@B.T@F1)/2
  L=scipy.sparse.csr_matrix(L)
  print('L')
  e, u=scipy.sparse.linalg.eigs(L, num, which='LR')
  A=nx.to_scipy_sparse_array(Gr)
  B=(A.T/A.sum(1)).T
  er, ur = scipy.sparse.linalg.eigs(B, k=1, which='LR')
  S = np.abs(ur).squeeze()
  S=S/S.sum()
  F=scipy.sparse.diags(S)
  F1=F.power(1/2)
  F2=F.power(-1/2)
  L=(F1@B@F2+F2@B.T@F1)/2
  L=scipy.sparse.csr_matrix(L)
  print("L")
  er, ur=scipy.sparse.linalg.eigs(L, num, which='LR')
  plt.figure(figsize = (10, 10), dpi=200)
  plt.scatter(e.real, e.imag, label='simulated')
  plt.savefig(path_out+name+to+'vals1.jpeg')
  plt.scatter(er.real, er.imag, label='real')
  # plt.axis('off')
  plt.legend()
  plt.show()
  plt.savefig(path_out+name+to+'c_vals1.jpeg')
  print('fin')

In [ ]:
def BH(G, Gr=None, to='', num=300, ret=True): #find bethe hessian eigs, return or compare if model
    r=np.sqrt(np.array(list(dict(G.degree()).values())).mean())
    B=nx.bethe_hessian_matrix(G.to_undirected(), r=r)
    e, u=scipy.sparse.linalg.eigs(B, 300, which='SR')
    if ret:
      k=np.where(e.real<0)[0]
      return u[:k]
    plt.figure(figsize = (10, 10), dpi=200)
    plt.scatter(e.real, e.imag, label='simulated')
    plt.savefig(path_out+name+to+'vals.jpeg')
    print('B')
    r=np.sqrt(np.array(list(dict(Gr.degree()).values())).mean())
    B=nx.bethe_hessian_matrix(Gr.to_undirected(), r=r)
    er, ur=scipy.sparse.linalg.eigs(B, num, which='SR')
    plt.scatter(er.real, er.imag, label='real')
    plt.legend()
    plt.show()
    plt.savefig(path_out+name+to+'c_vals.jpeg')
    print('B')

In [ ]:
def spectral_clustering(embedding, n_clusters=100):
    kmeans = KMeans(n_clusters=n_clusters, n_init=1000, max_iter=10000)
    kmeans.fit(embedding)
    return kmeans.labels_

## Models

In [ ]:
stat=calc_triangles(G) #calculate triangles distribution in real graph
x=pd.DataFrame(stat.T, columns=['t1', 't2', 't3', 't4'])
x.to_csv(path_out+name+'tri.csv')
stat=np.array(pd.read_csv(path_out+name+'self/tri.csv'))
tri=stat[-1, 1:]
tri

In [ ]:
x=np.arange(0, 1+0.1, 0.1)
plt.figure(figsize=(8, 6), dpi=200)
for i in range(4):
  plt.plot(x, stat[i], label=str(i+1)+' type')
plt.legend()
plt.savefig(path_out+name+'tri_stats.jpeg')

In [ ]:
num=1000
suf=''
tab=True
tab_name='models2.csv'

In [ ]:
tab=pd.read_csv(path_out+name+tab_name, index_col=0)
tab.columns=[int(i) if i in [str(j) for j in range(10)] else i for i in tab.columns]
tab

In [ ]:
def BA(n, m): #directed preferential attachment
  m0=max(int(m//n), 2)
  G=nx.DiGraph()
  G.add_node(0)
  for i in range(1, m0):
    G.add_edge(i, 0)
  # print(G.degree())
  for i in range(m0, n):
    p=np.array(list(dict(G.degree()).values()))
    p=p/p.sum()
    to=np.random.choice(list(G.nodes()), m0, False, p)
    for k in to:
      G.add_edge(i, k)
    if not i%(n//10):
      print(i)
  return G

In [ ]:
num=len(G)

In [ ]:
m=G.number_of_edges()
m=nx.density(G)*num**2
G_BA=BA(num, m)
nx.write_graphml(G_BA, path_out+name+'BA_'+str(num)+suf+'.graphml')
if not os.path.exists(path_out+name+'BA_'+str(num)+suf+'/'):
  os.makedirs(path_out+name+'BA_'+str(num)+suf+'/')
stat=calc_triangles(G_BA, 1, False)
x=pd.DataFrame(stat.T, columns=['t1', 't2', 't3', 't4'])
x.to_csv(path_out+name+'triBA_'+str(num)+suf+'.csv')
calculate_graph_features(G_BA, path_out+name+'BA_'+str(num)+suf+'/', tab, 'BA_'+str(num)+suf+'')
tab.to_csv(path_out+name+tab_name)
tab

In [ ]:
def SB(n, m, c_num, p_blocks, sizes=0): #stochastic block
  print(m)
  G=nx.DiGraph()
  if not isinstance(sizes, np.ndarray):
    sizes=np.array([int(n/c_num) for i in range(c_num)])
    sizes[-1]+=n-sizes.sum()
  st=0
  for i in range(0, c_num):
    for j in range(sizes[i]):
      G.add_node(st+j)
      G.nodes()[st+j]['block']=i
    st+=sizes[i]
  i=0
  while not G.number_of_edges()>=m:
    k=np.random.choice(G.nodes(), 1)[0]
    cl=G.nodes()[k]['block']
    p=np.array([p_blocks[cl][G.nodes()[j]['block']] for j in G.nodes()])
    p=p/p.sum()
    l=np.random.choice(list(G.nodes()), 1, p=p)[0]
    if not (k, l) in G.edges() and not k==l:
      G.add_edge(k, l)
      i+=1
      if not i%(m//20):
        print(i)
  return G

In [ ]:
G_ref = nx.read_graphml(path_out+"dutch/DL/31/clustered.graphml")

In [ ]:
G_ref = nx.read_graphml(path_out+"dutch/NB/clustered.graphml")

In [ ]:
G_ref = nx.read_graphml(path_out+"dutch/NB2/clustered.graphml")

In [ ]:
cl_best=np.array([G_ref.nodes[i]['clust'] for i in G_ref.nodes()]) #clustering to mimic
mk=len(set(cl_best))
cl_A=np.zeros((mk, mk))
cl_nn=np.zeros(mk)
ns=list(G_ref.nodes)
for i in G_ref.nodes:
  for j in G_ref.nodes:
    if (i, j) in G_ref.edges():
      cl_A[G_ref.nodes[i]['clust']][G_ref.nodes[j]['clust']]+=1
cl_nn=[len(np.where(cl_best==i)[0]) for i in range(mk)]
for i in range(mk):
  for j in range(mk):
    l=cl_nn[i]*cl_nn[j]
    if i==j:
      l=cl_nn[i]*(cl_nn[j]-1)
    if not l==0:
      cl_A[i][j]/=l

### SB

In [ ]:
num=len(G)
suf1='base10'
suf='_0_'+suf1
p=nx.density(G)
m=p*num**2
m=G.number_of_edges()
sizes=0
# sizes=np.array(cl_nn)
# cl_num=len(cl_nn)
cl_num=10

In [ ]:
G_SB=SB(num, m, cl_num, cl_A, sizes)
nx.write_graphml(G_SB, path_out+name+'SB_'+str(num)+suf+'.graphml')
if not os.path.exists(path_out+name+'SB_'+str(num)+suf+'/'):
  os.makedirs(path_out+name+'SB_'+str(num)+suf+'/')
# G_SB=nx.read_graphml(path_out+name+'SB_'+str(num)+suf+'.graphml')
stat=calc_triangles(G_SB, 1, False)
x=pd.DataFrame(stat.T, columns=['t1', 't2', 't3', 't4'])
x.to_csv(path_out+name+'triSB_'+str(num)+suf+'.csv')
calculate_graph_features(G_SB, path_out+name+'SB_'+str(num)+suf+'/', tab, 'SB_'+str(num)+suf+'', diam=True)
tab.to_csv(path_out+name+tab_name)
tab

In [ ]:
tab.to_csv(path_out+name+tab_name)

In [ ]:
def SB1(n, m1, c_num, p_blocks, sizes=0):
  G=nx.DiGraph()
  if not isinstance(sizes, np.ndarray):
    sizes=np.array([int(n/c_num) for i in range(c_num)])
    sizes[-1]+=n-sizes.sum()
  else:
    sizes=sizes.copy()
  # print(sizes)
  for i in range(0, c_num):
    G.add_node(i)
    G.nodes[i]['block']=i
    sizes[i]-=1
    if sizes[i]==0:
      continue
    for j in range(1, 3*m1+1):
      G.add_edge(i+c_num*j, i)
      G.nodes()[i]['block']=i
      G.nodes()[i+c_num*j]['block']=i
      sizes[i]-=1
      if sizes[i]==0:
        break
  # print(sizes)
  for i in range(len(G), n):
    if not i%(n//10):
      print(i)
    p=sizes/sizes.sum()
    bl=np.random.choice([i for i in range(c_num)], 1, p=p)[0] #choose block which needs nodes
    probs={}
    for j, k in G.edges(): #choose edges to form triangles (preferentially inside block, with very small probability outside)
      probs[(j, k)]=G.degree[j]+G.degree[k] if G.nodes[j]['block']==bl and G.nodes[k]['block']==bl else 1e-50
    m=list(probs.keys()) #do GEN1
    p=np.array(list(probs.values()))
    p=p/p.sum()
    t=np.random.choice([i for i in range(len(m))], 3*m1, False, p)
    for num, j in enumerate(t):
      k, l = m[j]
      if num%3==0:
        G.add_edge(i, k)
        G.add_edge(i, l)
      elif num%3==1:
        G.add_edge(k, i)
        G.add_edge(l, i)
      elif num%3==2:
        G.add_edge(k, i)
        G.add_edge(i, l)
    j=0
    G.nodes[i]['block']=bl
    sizes[bl]-=1
    while not j==6*m1:
      k=np.random.choice(G.nodes(), 1)[0]
      cl=G.nodes()[k]['block']
      p=np.array([p_blocks[cl][G.nodes()[j]['block']] for j in G.nodes()])
      p=p/p.sum()
      l=np.random.choice(list(G.nodes()), 1, p=p)[0]
      if not (k, l) in G.edges() and not k==l:
        G.add_edge(k, l)
        j+=1
  return G

In [ ]:
suf='_1_'+suf1
p=nx.density(G)
m1=int(max(1, round(num*p/12)))

In [ ]:
G_SB=SB1(num, m1, cl_num, cl_A)
nx.write_graphml(G_SB, path_out+name+'SB_'+str(num)+suf+'.graphml')
if not os.path.exists(path_out+name+'SB_'+str(num)+suf+'/'):
  os.makedirs(path_out+name+'SB_'+str(num)+suf+'/')
# G_SB=nx.read_graphml(path_out+name+'SB_'+str(num)+suf+'.graphml')
stat=calc_triangles(G_SB, 1, False)
x=pd.DataFrame(stat.T, columns=['t1', 't2', 't3', 't4'])
x.to_csv(path_out+name+'triSB_'+str(num)+suf+'.csv')
calculate_graph_features(G_SB, path_out+name+'SB_'+str(num)+suf+'/', tab, 'SB_'+str(num)+suf+'', diam=0)
tab.to_csv(path_out+name+tab_name)
tab

In [ ]:
tab.to_csv(path_out+name+tab_name)

In [ ]:
def SB2(n, m1, c_num, p_blocks, tri, sizes=0):
  G=nx.DiGraph()
  if not isinstance(sizes, np.ndarray):
    sizes=np.array([int(n/c_num) for i in range(c_num)])
    sizes[-1]+=n-sizes.sum()
  else:
    sizes=sizes.copy()
  st=0
  for i in range(0, c_num):
    G.add_node(i)
    G.nodes[i]['block']=i
    sizes[i]-=1
    if sizes[i]==0:
      continue
    for j in range(1, 4*m1+1):
      G.add_edge(i+c_num*j, i)
      G.nodes()[i]['block']=i
      G.nodes()[i+c_num*j]['block']=i
      sizes[i]-=1
      if sizes[i]==0:
        break
  for i in range(len(G), n):
    if not i%(n//10):
      print(i)
    p=sizes/sizes.sum()
    bl=np.random.choice([i for i in range(c_num)], 1, p=p)[0]
    probs={}
    for j, k in G.edges():
      probs[(j, k)]=G.degree[j]+G.degree[k] if G.nodes[j]['block']==bl and G.nodes[k]['block']==bl else 1e-50
    m=list(probs.keys())
    p=np.array(list(probs.values()))
    p=p/p.sum()
    t=np.random.choice([i for i in range(len(m))], 4*m1, False, p)
    for j in t:
      num=np.random.choice([i for i in range(4)], 1, p=tri)[0]
      k, l = m[j]
      if num==0:
        G.add_edge(l, i)
        G.add_edge(i, k)
      elif num==1:
        G.add_edge(i, k)
        G.add_edge(i, l)
      elif num==2:
        G.add_edge(k, i)
        G.add_edge(l, i)
      elif num==3:
        G.add_edge(k, i)
        G.add_edge(i, l)
    j=0
    G.nodes[i]['block']=bl
    sizes[bl]-=1
    while not j==4*m1:
      k=np.random.choice(G.nodes(), 1)[0]
      cl=G.nodes()[k]['block']
      p=np.array([p_blocks[cl][G.nodes()[j]['block']] for j in G.nodes()])
      p=p/p.sum()
      l=np.random.choice(list(G.nodes()), 1, p=p)[0]
      if not (k, l) in G.edges() and not k==l:
        G.add_edge(k, l)
        j+=1
  return G

In [ ]:
suf='_2_'+suf1

In [ ]:
G_SB=SB2(num, m1, cl_num, cl_A, tri, sizes)
nx.write_graphml(G_SB, path_out+name+'SB_'+str(num)+suf+'.graphml')
if not os.path.exists(path_out+name+'SB_'+str(num)+suf+'/'):
  os.makedirs(path_out+name+'SB_'+str(num)+suf+'/')
# G_SB=nx.read_graphml(path_out+name+'SB_'+str(num)+suf+'.graphml')
stat=calc_triangles(G_SB, 1, False)
x=pd.DataFrame(stat.T, columns=['t1', 't2', 't3', 't4'])
x.to_csv(path_out+name+'triSB_'+str(num)+suf+'.csv')
calculate_graph_features(G_SB, path_out+name+'SB_'+str(num)+suf+'/', tab, 'SB_'+str(num)+suf+'', diam=True)
tab.to_csv(path_out+name+tab_name)
tab

In [ ]:
tab.to_csv(path_out+name+tab_name)

In [ ]:
def SB3(n, m1, c_num, p_blocks, tri, sizes=0, load=False, file='net.graphml'):
  G=nx.DiGraph()
  if load:
    G=nx.read_graphml(file)
  else:
    if not isinstance(sizes, np.ndarray):
      sizes=np.array([int(n/c_num) for i in range(c_num)])
      sizes[-1]+=n-sizes.sum()
    else:
      sizes=sizes.copy()
    for i in range(0, c_num):
      G.add_node(i)
      G.nodes[i]['block']=i
      sizes[i]-=1
      if sizes[i]==0:
        continue
      for j in range(1, 4*m1+1):
        G.add_edge(i+c_num*j, i)
        G.nodes()[i]['block']=i
        G.nodes()[i+c_num*j]['block']=i
        sizes[i]-=1
        if sizes[i]==0:
          break
  tri_use=tri.copy()
  for i in range(len(G), n):
    if not i%(n//10):
      nx.write_graphml(G, file)
      print(i)
    if load:
      i=str(i)
    p=sizes/sizes.sum()
    bl=np.random.choice([i for i in range(c_num)], 1, p=p)[0]
    probs={}
    for j, k in G.edges():
      probs[(j, k)]=G.degree[j]+G.degree[k] if G.nodes[j]['block']==bl and G.nodes[k]['block']==bl else 1e-50
    m=list(probs.keys())
    p=np.array(list(probs.values()))
    p=p/p.sum()
    t=np.random.choice([i for i in range(len(m))], 4*m1, False, p)
    for j in t:
      num=np.random.choice([i for i in range(4)], 1, p=tri_use)[0]
      k, l = m[j]
      if num==0:
        G.add_edge(l, i)
        G.add_edge(i, k)
      elif num==1:
        G.add_edge(i, k)
        G.add_edge(i, l)
      elif num==2:
        G.add_edge(k, i)
        G.add_edge(l, i)
      elif num==3:
        G.add_edge(k, i)
        G.add_edge(i, l)
    j=0
    G.nodes[i]['block']=bl
    sizes[bl]-=1
    while not j==4*m1:
      k=np.random.choice(G.nodes(), 1)[0]
      cl=G.nodes()[k]['block']
      p=np.array([p_blocks[cl][G.nodes()[j]['block']] for j in G.nodes()])
      p=p/p.sum()
      l=np.random.choice(list(G.nodes()), 1, p=p)[0]
      if not (k, l) in G.edges() and not k==l:
        G.add_edge(k, l)
        j+=1
    if not int(i)%(n//100):
      tri_upd=calc_triangles(G, 1, False)[:,0]
      tri_use=np.array([max(0, i) for i in 2*tri-tri_upd])
      tri_use=tri_use/tri_use.sum()
  return G

In [ ]:
suf='_3_'+suf1

In [ ]:
G_SB=SB3(num, m1, cl_num, cl_A, tri, sizes, file=path_out+name+'SB_'+str(num)+suf+'.graphml', load=True)
nx.write_graphml(G_SB, path_out+name+'SB_'+str(num)+suf+'.graphml')
if not os.path.exists(path_out+name+'SB_'+str(num)+suf+'/'):
  os.makedirs(path_out+name+'SB_'+str(num)+suf+'/')
# G_SB=nx.read_graphml(path_out+name+'SB_'+str(num)+suf+'.graphml')
stat=calc_triangles(G_SB, 1, False)
x=pd.DataFrame(stat.T, columns=['t1', 't2', 't3', 't4'])
x.to_csv(path_out+name+'triSB_'+str(num)+suf+'.csv')
calculate_graph_features(G_SB, path_out+name+'SB_'+str(num)+suf+'/', tab, 'SB_'+str(num)+suf+'', diam=1)
tab.to_csv(path_out+name+tab_name)
tab

In [ ]:
tab.to_csv(path_out+name+tab_name)

### GEN

In [ ]:
p=nx.density(G)
m1=max(1, round(num*p/12))
m1
G_gen=gen(num, m1, p)
nx.write_graphml(G_gen, path_out+name+'GEN1_'+str(num)+suf+'.graphml')
stat=calc_triangles(G_gen, 1, False)
x=pd.DataFrame(stat.T, columns=['t1', 't2', 't3', 't4'])
x.to_csv(path_out+name+'triGEN1_'+str(num)+suf+'.csv')
if not tab:  
  tab=pd.DataFrame(columns=['nn', 'ne', 'diameter', 'radius', 'ASPL', 'transitivity', 'ACC', 'GCN', 'Density', 'Degree mean',
                            'Degree var', 'In-degree mean', 'In-degree var', 'Out-degree mean', 'Out-degree var',
                            'gamma', 'k_min', 'AII', 'AOO', "AIO", 'AOI', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
  if not os.path.exists(path_out+name+'self/'):
    os.makedirs(path_out+name+'self/')
  calculate_graph_features(G, path_out+name+'self/', tab, 'ORIG', diam=False)
else:
  tab=pd.read_csv(path_out+name+tab_name)
  tab.columns=[int(i) if i in [str(j) for j in range(10)] else i for i in tab.columns]
if not os.path.exists(path_out+name+'GEN1_'+str(num)+suf+'/'):
  os.makedirs(path_out+name+'GEN1_'+str(num)+suf+'/')
calculate_graph_features(G_gen, path_out+name+'GEN1_'+str(num)+suf+'/', tab, 'GEN1_'+str(num)+suf+'')

In [ ]:
G_ER=nx.erdos_renyi_graph(num, nx.density(G), directed=True)
nx.write_graphml(G_ER, path_out+name+'ER_'+str(num)+suf+'.graphml')
if not os.path.exists(path_out+name+'ER_'+str(num)+suf+'/'):
  os.makedirs(path_out+name+'ER_'+str(num)+suf+'/')
stat=calc_triangles(G_ER, 1, False)
x=pd.DataFrame(stat.T, columns=['t1', 't2', 't3', 't4'])
x.to_csv(path_out+name+'triER_'+str(num)+suf+'.csv')
calculate_graph_features(G_ER, path_out+name+'ER_'+str(num)+suf+'/', tab, 'ER_'+str(num)+suf+'')
tab.to_csv(path_out+name+tab_name)

In [ ]:
m1=max(1, round(num*p/12))
m1

In [ ]:
G_gen2=gen2(num, tri, m1)
nx.write_graphml(G_gen2, path_out+name+'GEN2_'+str(num)+suf+'.graphml')
stat=calc_triangles(G_gen2, 1, False)
x=pd.DataFrame(stat.T, columns=['t1', 't2', 't3', 't4'])
x.to_csv(path_out+name+'triGEN2_'+str(num)+suf+'.csv')
if not os.path.exists(path_out+name+'GEN2_'+str(num)+suf+'/'):
  os.makedirs(path_out+name+'GEN2_'+str(num)+suf+'/')
G_gen2=nx.read_graphml(path_out+name+'GEN2_'+str(num)+suf+'.graphml')
calculate_graph_features(G_gen2, path_out+name+'GEN2_'+str(num)+suf+'/', tab, 'GEN2_'+str(num)+suf+'')

In [ ]:
tab.to_csv(path_out+name+tab_name)

In [ ]:
G_gen3=gen3(num, tri, 0, path_out+name+'GEN3_'+str(num)+suf+'.graphml')
nx.write_graphml(G_gen3, path_out+name+'GEN3_'+str(num)+suf+'.graphml')
stat=calc_triangles(G_gen3, 1, False)
print(stat)
x=pd.DataFrame(stat.T, columns=['t1', 't2', 't3', 't4'])
x.to_csv(path_out+name+'triGEN3_'+str(num)+suf+'.csv')
if not os.path.exists(path_out+name+'GEN3_'+str(num)+suf+'/'):
  os.makedirs(path_out+name+'GEN3_'+str(num)+suf+'/')
calculate_graph_features(G_gen3, path_out+name+'GEN3_'+str(num)+suf+'/', tab, 'GEN3_'+str(num)+suf+'')

In [ ]:
tab.to_csv(path_out+name+tab_name)

## Centralities

In [ ]:
tab=pd.DataFrame(columns=[i for i in range(15)])
tab
for num in ['rus', 'en_r1', 'en_r123', 'dutch', 'USF']:
  G = nx.read_graphml(path_out+num+"_main.graphml")
  ec=nx.eigenvector_centrality(G, max_iter=10000)
  plt.figure(figsize=(8, 6), dpi=200) 
  plt.title("Eigenvector centrality distribution")
  plt.xlabel("Centrality")
  plt.ylabel("Number of nodes")
  plt.hist(list(ec.values()), bins=100)
  plt.savefig(path_out+num+'/ec.png')
  plt.close('All')
  print(num)
  for m, j in enumerate(sorted(ec, key=ec.get, reverse=True)[:15]):
    print(G.nodes()[j]['name'], ' -- ', round(ec[j], 4), end=', ')
    tab.loc[num, m]=G.nodes()[j]['name']
  print()
tab.to_csv(path_out+'evc.csv')
tab

In [ ]:
tab=pd.DataFrame(columns=[i for i in range(15)])
for num in ['rus', 'en_r1', 'en_r123', 'dutch', 'USF']:
  G = nx.read_graphml(path_out+num+"_main.graphml")
  ec=nx.degree_centrality(G)
  plt.figure(figsize=(8, 6), dpi=200) 
  plt.title("Degree centrality distribution")
  plt.xlabel("Centrality")
  plt.ylabel("Number of nodes")
  plt.hist(list(ec.values()), bins=100)
  plt.savefig(path_out+num+'/dc.png')
  plt.close('All')
  print(num)
  for m, j in enumerate(sorted(ec, key=ec.get, reverse=True)[:15]):
    print(G.nodes()[j]['name'], ' -- ', round(ec[j], 4), end=', ')
    tab.loc[num, m]=G.nodes()[j]['name']
  print()
tab.to_csv(path_out+'dc.csv')
tab

In [ ]:
tab=pd.DataFrame(columns=[i for i in range(15)])
for num in ['rus', 'en_r1', 'en_r123', 'dutch', 'USF']:
  G = nx.read_graphml(path_out+num+"_main.graphml")
  ec=nx.in_degree_centrality(G)
  plt.figure(figsize=(8, 6), dpi=200) 
  plt.title("In-degree centrality distribution")
  plt.xlabel("Centrality")
  plt.ylabel("Number of nodes")
  plt.hist(list(ec.values()), bins=100)
  plt.savefig(path_out+num+'/idc.png')
  plt.close('All')
  print(num)
  for m, j in enumerate(sorted(ec, key=ec.get, reverse=True)[:15]):
    print(G.nodes()[j]['name'], ' -- ', round(ec[j], 4), end=', ')
    tab.loc[num, m]=G.nodes()[j]['name']
  print()
tab.to_csv(path_out+'idc.csv')
tab

In [ ]:
tab=pd.DataFrame(columns=[i for i in range(15)])
for num in ['rus', 'en_r1', 'en_r123', 'dutch', 'USF']:
  G = nx.read_graphml(path_out+num+"_main.graphml")
  ec=nx.out_degree_centrality(G)
  plt.figure(figsize=(8, 6), dpi=200) 
  plt.title("Out-degree centrality distribution")
  plt.xlabel("Centrality")
  plt.ylabel("Number of nodes")
  plt.hist(list(ec.values()), bins=100)
  plt.savefig(path_out+num+'/odc.png')
  plt.close('All')
  print(num)
  for m, j in enumerate(sorted(ec, key=ec.get, reverse=True)[:15]):
    print(G.nodes()[j]['name'], ' -- ', round(ec[j], 4), end=', ')
    tab.loc[num, m]=G.nodes()[j]['name']
  print()
tab.to_csv(path_out+'odc.csv')
tab

In [ ]:
tab=pd.DataFrame(columns=[i for i in range(15)])
for num in ['rus', 'en_r1', 'en_r123', 'dutch', 'USF']:
  G = nx.read_graphml(path_out+num+"_main.graphml")
  ec=nx.closeness_centrality(G)
  plt.figure(figsize=(8, 6), dpi=200) 
  plt.title("Closeness centrality distribution")
  plt.xlabel("Centrality")
  plt.ylabel("Number of nodes")
  plt.hist(list(ec.values()), bins=100)
  plt.savefig(path_out+num+'/cc.png')
  plt.close('All')
  print(num)
  for m, j in enumerate(sorted(ec, key=ec.get, reverse=True)[:15]):
    print(G.nodes()[j]['name'], ' -- ', round(ec[j], 4), end=', ')
    tab.loc[num, m]=G.nodes()[j]['name']
  print()
tab.to_csv(path_out+'cc.csv')
tab

In [ ]:
tab=pd.DataFrame(columns=[i for i in range(15)])
for num in ['rus', 'en_r1', 'en_r123', 'dutch', 'USF']:
  G = nx.read_graphml(path_out+num+"_main.graphml")
  ec=nx.betweenness_centrality(G)
  plt.figure(figsize=(8, 6), dpi=200) 
  plt.title("Betweenness centrality distribution")
  plt.xlabel("Centrality")
  plt.ylabel("Number of nodes")
  plt.hist(list(ec.values()), bins=100)
  plt.savefig(path_out+num+'/bc.png')
  plt.close('All')
  print(num)
  for m, j in enumerate(sorted(ec, key=ec.get, reverse=True)[:15]):
    print(G.nodes()[j]['name'], ' -- ', round(ec[j], 4), end=', ')
    tab.loc[num, m]=G.nodes()[j]['name']
  print()
tab.to_csv(path_out+'bc.csv')
tab

In [ ]:
for num in ['rus', 'USF', 'en_r1', 'en_r123', 'dutch']:
  # G = nx.read_graphml(path_out+num+"_main.graphml")
  for j in ["/ER", "/GEN", "/GEN2", "/GEN3"]:
    for k in ["", "_2", "_100", "_1000", "_2000"]:
      name=path_out+num+j+k
      if (j=="/ER" or num=="USF") and k=='_2':
        continue
      if j=='/GEN' and not k=='':
        name=path_out+num+j+'1'+k
      if num=='/rus' and not k=='_2' and not j=="/ER":
        name=name+"_1"
      G=nx.read_graphml(name+'.graphml')
      ec=nx.eigenvector_centrality(G, max_iter=10000)
      plt.figure(figsize=(8, 6), dpi=200) 
      plt.title("Eigenvector centrality distribution")
      plt.xlabel("Centrality")
      plt.ylabel("Number of nodes")
      plt.hist(list(ec.values()), bins=100)
      plt.savefig(name+'_ec.png')
      plt.close('All')
      print(name)

In [ ]:
for num in ['rus', 'USF', 'en_r1', 'en_r123', 'dutch']:
  # G = nx.read_graphml(path_out+num+"_main.graphml")
  for j in ["/ER", "/GEN", "/GEN2", "/GEN3"]:
    for k in ["", "_2", "_100", "_1000", "_2000"]:
      name=path_out+num+j+k
      if (j=="/ER" or num=="USF") and k=='_2':
        continue
      if j=='/GEN' and not k=='':
        name=path_out+num+j+'1'+k
      if num=='/rus' and not k=='_2' and not j=="/ER":
        name=name+"_1"
      G=nx.read_graphml(name+'.graphml')
      ec=nx.degree_centrality(G)
      plt.figure(figsize=(8, 6), dpi=200) 
      plt.title("Degree centrality distribution")
      plt.xlabel("Centrality")
      plt.ylabel("Number of nodes")
      plt.hist(list(ec.values()), bins=100)
      plt.savefig(name+'_dc.png')
      plt.close('All')
      print(name)

In [ ]:
for num in ['rus', 'USF', 'en_r1', 'en_r123', 'dutch']:
  # G = nx.read_graphml(path_out+num+"_main.graphml")
  for j in ["/ER", "/GEN", "/GEN2", "/GEN3"]:
    for k in ["", "_2", "_100", "_1000", "_2000"]:
      name=path_out+num+j+k
      if (j=="/ER" or num=="USF") and k=='_2':
        continue
      if j=='/GEN' and not k=='':
        name=path_out+num+j+'1'+k
      if num=='/rus' and not k=='_2' and not j=="/ER":
        name=name+"_1"
      G=nx.read_graphml(name+'.graphml')
      ec=nx.in_degree_centrality(G)
      plt.figure(figsize=(8, 6), dpi=200) 
      plt.title("In-degree centrality distribution")
      plt.xlabel("Centrality")
      plt.ylabel("Number of nodes")
      plt.hist(list(ec.values()), bins=100)
      plt.savefig(name+'_idc.png')
      plt.close('All')
      print(name)

In [ ]:
for num in ['rus', 'USF', 'en_r1', 'en_r123', 'dutch']:
  # G = nx.read_graphml(path_out+num+"_main.graphml")
  for j in ["/ER", "/GEN", "/GEN2", "/GEN3"]:
    for k in ["", "_2", "_100", "_1000", "_2000"]:
      name=path_out+num+j+k
      if (j=="/ER" or num=="USF") and k=='_2':
        continue
      if j=='/GEN' and not k=='':
        name=path_out+num+j+'1'+k
      if num=='/rus' and not k=='_2' and not j=="/ER":
        name=name+"_1"
      G=nx.read_graphml(name+'.graphml')
      ec=nx.out_degree_centrality(G)
      plt.figure(figsize=(8, 6), dpi=200) 
      plt.title("Out-degree centrality distribution")
      plt.xlabel("Centrality")
      plt.ylabel("Number of nodes")
      plt.hist(list(ec.values()), bins=100)
      plt.savefig(name+'_odc.png')
      plt.close('All')
      print(name)

In [ ]:
for num in ['rus', 'USF', 'en_r1', 'en_r123', 'dutch']:
  # G = nx.read_graphml(path_out+num+"_main.graphml")
  for j in ["/ER", "/GEN", "/GEN2", "/GEN3"]:
    for k in ["", "_2", "_100", "_1000", "_2000"]:
      name=path_out+num+j+k
      if (j=="/ER" or num=="USF") and k=='_2':
        continue
      if j=='/GEN' and not k=='':
        name=path_out+num+j+'1'+k
      if num=='/rus' and not k=='_2' and not j=="/ER":
        name=name+"_1"
      G=nx.read_graphml(name+'.graphml')
      ec=nx.closeness_centrality(G)
      plt.figure(figsize=(8, 6), dpi=200) 
      plt.title("Closeness centrality distribution")
      plt.xlabel("Centrality")
      plt.ylabel("Number of nodes")
      plt.hist(list(ec.values()), bins=100)
      plt.savefig(name+'_cc.png')
      plt.close('All')
      print(name)

In [ ]:
for num in ['rus', 'USF', 'en_r1', 'en_r123', 'dutch']:
  # G = nx.read_graphml(path_out+num+"_main.graphml")
  for j in ["/ER", "/GEN", "/GEN2", "/GEN3"]:
    for k in ["", "_2", "_100", "_1000", "_2000"]:
      name=path_out+num+j+k
      if (j=="/ER" or num=="USF") and k=='_2':
        continue
      if j=='/GEN' and not k=='':
        name=path_out+num+j+'1'+k
      if num=='/rus' and not k=='_2' and not j=="/ER":
        name=name+"_1"
      G=nx.read_graphml(name+'.graphml')
      ec=nx.betweenness_centrality(G)
      plt.figure(figsize=(8, 6), dpi=200) 
      plt.title("Betweenness centrality distribution")
      plt.xlabel("Centrality")
      plt.ylabel("Number of nodes")
      plt.hist(list(ec.values()), bins=100)
      plt.savefig(name+'_bc.png')
      plt.close('All')
      print(name)

## DL & BH pictures

In [ ]:
G_gen2 = nx.read_graphml(path_out+name+"GEN2_2.graphml")
G_gen3 = nx.read_graphml(path_out+name+"GEN3_2.graphml")
G_ER = nx.read_graphml(path_out+name+"ER.graphml")
G_gen = nx.read_graphml(path_out+name+"GEN1_2.graphml")

In [ ]:
if not os.path.exists(path_out+name+'gen1_2DL/'):
  os.makedirs(path_out+name+'gen1_2DL/')
DL_2(G_gen, G, 'gen1_2DL/')
if not os.path.exists(path_out+name+'gen2_2DL/'):
  os.makedirs(path_out+name+'gen2_2DL/')
DL_2(G_gen2.subgraph(max(nx.strongly_connected_components(G_gen2), key=len)), G, 'gen2_2DL/')
if not os.path.exists(path_out+name+'gen3_2DL/'):
  os.makedirs(path_out+name+'gen3_2DL/')
# DL(G_gen3, 'gen3_DL/')
DL_2(G_gen3.subgraph(max(nx.strongly_connected_components(G_gen3), key=len)), G, 'gen3_DL/')
if not os.path.exists(path_out+name+'ER_DL/'):
  os.makedirs(path_out+name+'ER_DL/')
DL_2(G_ER, G, 'ER_DL/')

In [ ]:
if not os.path.exists(path_out+name+'gen1_2BH/'):
  os.makedirs(path_out+name+'gen1_2BH/')
BH(G_gen, G, 'gen1_2BH/')
if not os.path.exists(path_out+name+'gen2_2BH/'):
  os.makedirs(path_out+name+'gen2_2BH/')
BH(G_gen2.subgraph(max(nx.strongly_connected_components(G_gen2), key=len)), G, 'gen2_2BH/')
if not os.path.exists(path_out+name+'gen3_2BH/'):
  os.makedirs(path_out+name+'gen3_2BH/')
BH(G_gen3.subgraph(max(nx.strongly_connected_components(G_gen3), key=len)), G, 'gen3_2BH/')
if not os.path.exists(path_out+name+'ER_BH/'):
  os.makedirs(path_out+name+'ER_BH/')
BH(G_ER, G, 'ER_BH/')

In [ ]:
name='USF/SB_4845_3_base500'
G = nx.read_graphml(path_out+name+".graphml")
# name=name+'GEN1_2/'

In [ ]:
if not os.path.exists(path_out+name+'/DL2/'):
  os.makedirs(path_out+name+'/DL2/')
A=nx.to_scipy_sparse_array(G)
B=(A.T/A.sum(1)).T
B[B==np.inf]=0
B[np.isnan(B)]=0
e = scipy.sparse.linalg.eigs(B, k=A.shape[0], which='LR', return_eigenvectors=False)
plt.figure(figsize = (10, 10), dpi=200)
plt.scatter(e.real, e.imag)
plt.xlim(-1,1)
plt.ylim(-1,1)
# plt.axis('off')
plt.show()
plt.savefig(path_out+name+'/DL2/'+'vals.jpeg')
print(name)

## NB

In [ ]:
which='USF_main'
name='USF/SB_2000_2_base200'
G = nx.read_graphml(path_out+name+".graphml")

In [ ]:
!rm -r el_lib

In [ ]:
if not os.path.exists(path_out+name+'/NB2/'):
  os.makedirs(path_out+name+'/NB2/')

In [ ]:
G=nx.convert_node_labels_to_integers(G)

In [ ]:
NB(G, save_to=path_out+name+'/NB2/', num=3000, dir=True, LM=True)
print(name)

In [ ]:
shutil.copytree(path_in+'el_lib', 'el_lib')
from el_lib import Alpha
pipeline_object = Alpha(G)

In [ ]:
adj_sym=nx.to_numpy_array(G, weight=None) #non-symmetrized
graph_sym = nx.from_numpy_array(adj_sym, create_using=nx.DiGraph)
all_edg_sym = np.array(graph_sym.edges())
A=scipy.sparse.load_npz(path_out+name+'NB2/NB_A.npz')

In [ ]:
#symmetrized
adj = pipeline_object.adjacency_mat()
_, adj_sym = pipeline_object.preprocessing_matrix(adj)
all_edg_sym = pipeline_object.edges_extracting(adj_sym)
A=scipy.sparse.load_npz(path_out+name+'NB/NB_A.npz')

In [ ]:
vals_flow, vecs =  scipy.sparse.linalg.eigs(A, k = 40,  which='LR', return_eigenvectors=True)
tail=True
degrees = np.sum(adj_sym, axis=1)
cr_rad = np.sqrt(np.mean(np.array(degrees)/(np.array([x-1 if x>1 else x for x in degrees])))/(np.mean(degrees)))
if tail == True: 
    eig_vals = vals_flow[vals_flow > cr_rad]
order = np.argsort(-np.abs(np.array(eig_vals)))

vecs = vecs[:,order[1:]]
vals = np.array(eig_vals)[order[1:]] 

len_of_tail = np.shape(vals)[0]

translated_eig_vec = np.zeros((np.shape(adj_sym)[0],len(vals)))
for i in range(len(all_edg_sym)):
    for k in range(len(vals)):
        translated_eig_vec[all_edg_sym[i][0],k] += vecs[i,k]
cr_rad, len_of_tail

In [ ]:
colours = np.zeros((np.shape(translated_eig_vec)[1], np.shape(translated_eig_vec)[0]))

for i in np.arange(1, np.shape(translated_eig_vec)[1]+1):
    print(i, end=' ')
    if np.shape(translated_eig_vec)[1] != 0:
    
        cluster = KMeans(n_clusters=i+1, n_init=50, max_iter = 5000)
        cluster.fit(translated_eig_vec[:,:i+1])
        cluster = pipeline_object.sorted_cluster(translated_eig_vec[:, :i+1], cluster)
        colours[i-1,:] = cluster.labels_
      
    else:  
        colours[i-1,:] = [0] * np.shape(translated_eig_vec)[0] 
    if not i%5 or i==np.shape(translated_eig_vec)[1]:
      print('\n', i-5+1, i+1)
      c_in, c_out, w_in, w_out, c, optimal_clusters = pipeline_object.cluster_limit(colours[i-5:i], adj_sym)
      for j in range(np.shape(optimal_clusters)[0]):
        print((i-5+j+2))
        if optimal_clusters[j,0] > optimal_clusters[j,1]/(j+2)*(i-5+j+2):
            print(f'Number of clusters {int(optimal_clusters[j,2])+i-5}: {round(optimal_clusters[j,0],3)} > {round(optimal_clusters[j,1]/(j+2)*(i-5+j+2),3)}')
        else:
            print(f'Number of clusters {int(optimal_clusters[j,2])+i-5}: {round(optimal_clusters[j,0],3)} < {round(optimal_clusters[j,1]/(j+2)*(i-5+j+2),3)}')

In [ ]:
c=7
cluster = KMeans(n_clusters=c, n_init=100, max_iter = 10000)
cluster.fit(translated_eig_vec[:, :c])
cluster = pipeline_object.sorted_cluster(translated_eig_vec, cluster)
cl_best= cluster.labels_
ns=list(G.nodes)
k_best=c
for i in range(len(G)):
    G.nodes[ns[i]]['clust']=cl_best[i]
for i in range(k_best):
    clust=np.where(cl_best==i)[0]
    nodes=[ns[j] for j in clust]
    print('\n\n', i, len(clust))
    if len(clust)<500:
        for p in nodes:
            print(G.nodes()[p]['name'], end=', ')
nx.write_graphml(G, path_out+name+'NB/clustered.graphml')

## DL & BH clusters

In [ ]:
if not os.path.exists(path_out+name):
  os.makedirs(path_out+name)

In [ ]:
mk=300
which='BH'
E=BH(G)[:, 1:]
mk=E.shape[1] if which=="BH" else mk
ns=list(G.nodes)
st=mk
n=len(G)
ed=G.number_of_edges()
M=nx.directed_modularity_matrix(G)
mod_top=-2
flag=0
for k in range(st, mk+1):
  if k<2:
    break
  print('\n\n', k)
  emb=((E.T)[:k]).T
  cl=spectral_clustering(emb, k)
  I=np.zeros((n, n))
  for i in range(n):
    for j in range(i, n):
      I[i][j]=I[j][i]=int(cl[i]==cl[j])
  mod=(M*I).sum()/ed
  if mod>mod_top:
    mod_top=mod
    print(mod, '-- new best modularity')
    flag=1
    cl_best=cl
    k_best=k
  if flag and not k%5 or st==mk:
    flag=0
    print(k_best)
    for i in range(len(G)):
        G.nodes[ns[i]]['clust']=cl_best[i]
    if not os.path.exists(path_out+name+which+'/'+str(k_best)+'/'):
        os.makedirs(path_out+name+which+'/'+str(k_best)+'/')
    nx.write_graphml(G, path_out+name+which+'/'+str(k_best)+'/clustered.graphml')

## NB

In [ ]:
which='USF_main'
name='USF/'
G = nx.read_graphml(path_out+"graphs/"+which+".graphml")

In [ ]:
NB(G, save_to=path_out+name+'NB2/', dir=True)

In [ ]:
shutil.copytree(path_in+'el-lib', 'el_lib')
from el_lib import Alpha
pipeline_object = Alpha(G)

In [ ]:
adj_sym=nx.to_numpy_array(G, weight=None) #non-symmetrized
graph_sym = nx.from_numpy_array(adj_sym, create_using=nx.DiGraph)
all_edg_sym = np.array(graph_sym.edges())
A=scipy.sparse.load_npz(path_out+name+'NB2/NB_A.npz')

In [ ]:
#symmetrized
adj = pipeline_object.adjacency_mat()
_, adj_sym = pipeline_object.preprocessing_matrix(adj)
all_edg_sym = pipeline_object.edges_extracting(adj_sym)
A=scipy.sparse.load_npz(path_out+name+'NB/NB_A.npz')

In [ ]:
vals_flow, vecs =  scipy.sparse.linalg.eigs(A, k = 40,  which='LR', return_eigenvectors=True)
tail=True
degrees = np.sum(adj_sym, axis=1)
cr_rad = np.sqrt(np.mean(np.array(degrees)/(np.array([x-1 if x>1 else x for x in degrees])))/(np.mean(degrees)))
if tail == True: 
    eig_vals = vals_flow[vals_flow > cr_rad]
order = np.argsort(-np.abs(np.array(eig_vals)))

vecs = vecs[:,order[1:]]
vals = np.array(eig_vals)[order[1:]] 

len_of_tail = np.shape(vals)[0]

translated_eig_vec = np.zeros((np.shape(adj_sym)[0],len(vals)))
for i in range(len(all_edg_sym)):
    for k in range(len(vals)):
        translated_eig_vec[all_edg_sym[i][0],k] += vecs[i,k]
cr_rad, len_of_tail

In [ ]:
colours = np.zeros((np.shape(translated_eig_vec)[1], np.shape(translated_eig_vec)[0]))

for i in np.arange(1, np.shape(translated_eig_vec)[1]+1):
    print(i, end=' ')
    if np.shape(translated_eig_vec)[1] != 0:
    
        cluster = KMeans(n_clusters=i+1, n_init=50, max_iter = 5000)
        cluster.fit(translated_eig_vec[:,:i+1])
        cluster = pipeline_object.sorted_cluster(translated_eig_vec[:, :i+1], cluster)
        colours[i-1,:] = cluster.labels_
      
    else:  
        colours[i-1,:] = [0] * np.shape(translated_eig_vec)[0] 
    if not i%5 or i==np.shape(translated_eig_vec)[1]:
      print('\n', i-5+1, i+1)
      c_in, c_out, w_in, w_out, c, optimal_clusters = pipeline_object.cluster_limit(colours[i-5:i], adj_sym)
      for j in range(np.shape(optimal_clusters)[0]):
        print((i-5+j+2))
        if optimal_clusters[j,0] > optimal_clusters[j,1]/(j+2)*(i-5+j+2):
            print(f'Number of clusters {int(optimal_clusters[j,2])+i-5}: {round(optimal_clusters[j,0],3)} > {round(optimal_clusters[j,1]/(j+2)*(i-5+j+2),3)}')
        else:
            print(f'Number of clusters {int(optimal_clusters[j,2])+i-5}: {round(optimal_clusters[j,0],3)} < {round(optimal_clusters[j,1]/(j+2)*(i-5+j+2),3)}')

In [ ]:
c=7
cluster = KMeans(n_clusters=c, n_init=100, max_iter = 10000)
cluster.fit(translated_eig_vec[:, :c])
cluster = pipeline_object.sorted_cluster(translated_eig_vec, cluster)
cl_best= cluster.labels_
ns=list(G.nodes)
k_best=c
for i in range(len(G)):
    G.nodes[ns[i]]['clust']=cl_best[i]
for i in range(k_best):
    clust=np.where(cl_best==i)[0]
    nodes=[ns[j] for j in clust]
    print('\n\n', i, len(clust))
    if len(clust)<500:
        for p in nodes:
            print(G.nodes()[p]['name'], end=', ')
nx.write_graphml(G, path_out+name+'NB/clustered.graphml')

## Clusters

In [ ]:
G = nx.read_graphml(path_out+"USF/DL/269/clustered.graphml") #!load only one that needed ignoring others
name='USF/DL/'

In [ ]:
G = nx.read_graphml(path_out+"dutch/DL/31/clustered.graphml")
name='dutch/DL/'

In [ ]:
G = nx.read_graphml(path_out+"en_r1/DL/53/clustered.graphml")
name='en_r1/DL/'

In [ ]:
G = nx.read_graphml(path_out+"en_r123/DL/22/clustered.graphml")
name='en_r123/DL/'

In [ ]:
G = nx.read_graphml(path_out+"rus/DL/21/clustered.graphml")
name='rus/DL/'

In [ ]:
G = nx.read_graphml(path_out+"rus/BHr+/18/clustered.graphml")
name='rus/BHr+/'

In [ ]:
G = nx.read_graphml(path_out+"en_r123/NB/clustered.graphml")
name='en_r123/NB/'

In [ ]:
G = nx.read_graphml(path_out+"dutch/NB/clustered.graphml")
name='dutch/NB/'

In [ ]:
G = nx.read_graphml(path_out+"USF/NB2/clustered.graphml")
name='USF/NB2/'

In [ ]:
G = nx.read_graphml(path_out+"rus/NB2/clustered.graphml")
name='rus/NB2/'

In [ ]:
G = nx.read_graphml(path_out+"en_r1/NB2/clustered.graphml")
name='en_r1/NB2/'

In [ ]:
G = nx.read_graphml(path_out+"en_r123/NB2/clustered.graphml")
name='en_r123/NB2/'

In [ ]:
G = nx.read_graphml(path_out+"dutch/NB2/clustered.graphml")
name='dutch/NB2/'

In [ ]:
cl_best=np.array([G.nodes[i]['clust'] for i in G.nodes()])
mk=len(set(cl_best))
cl_A=np.zeros((mk, mk))
cl_nn=np.zeros(mk)
ns=list(G.nodes)
for i in G.nodes:
  for j in G.nodes:
    if (i, j) in G.edges():
      cl_A[G.nodes[i]['clust']][G.nodes[j]['clust']]+=1
cl_nn=[len(np.where(cl_best==i)[0]) for i in range(mk)]
for i in range(mk):
  for j in range(mk):
    l=cl_nn[i]*cl_nn[j]
    if i==j:
      l=cl_nn[i]*(cl_nn[j]-1)
    if not l==0:
      cl_A[i][j]/=l
    # if i==j and cl_A[i][j]==0 and l==0:
    #   cl_A[i][j]=1
G_cl=nx.DiGraph(cl_A-np.diag(np.diagonal(cl_A)))

In [ ]:
%matplotlib inline

In [ ]:
name

In [ ]:
plt.figure(figsize=(12, 12),dpi=200)
r=sns.heatmap(cl_A)
plt.title("Clustering density matrix")
plt.savefig(path_out+name+'matr.png')
plt.show()
plt.close('All')

In [ ]:
ed=G.number_of_edges()
M=nx.directed_modularity_matrix(G)
cl=[G.nodes()[i]['clust'] for i in G.nodes()]
n=len(G)
I=np.zeros((n, n))
for i in range(n):
  for j in range(i, n):
    I[i][j]=I[j][i]=int(cl[i]==cl[j])
mod=(M*I).sum()/ed
mod

In [ ]:
import sklearn
num='USF/DL/269/'
num2='USF/NB2/'
G = nx.read_graphml(path_out+num+"clustered.graphml") 
G2 = nx.read_graphml(path_out+num2+"clustered.graphml")
print(num, num2)
print(G.nodes(), '\n', G2.nodes())
# cor=list(enumerate(G2.nodes()))
cor=[(int(i), G2.nodes()[i]['name']) for i in G2.nodes()]
l1=[G2.nodes()[i]['clust'] for i in G2.nodes()]
l2=[G.nodes()[i]['clust'] for _, i in cor]
sc=sklearn.metrics.normalized_mutual_info_score(l1, l2)
ar=sklearn.metrics.adjusted_rand_score(l1, l2)
sc, ar

In [ ]:

cor=[(int(i), G2.nodes()[i]['name']) for i in G2.nodes()]

In [ ]:
tab=pd.DataFrame(columns=['nn', 'ne', 'diameter', 'radius', 'ASPL', 'transitivity', 'ACC', 'GCN', 'Density', 'Degree mean',
                          'Degree var', 'In-degree mean', 'In-degree var', 'Out-degree mean', 'Out-degree var',
                          'gamma', 'k_min', 'AII', 'AOO', "AIO", 'AOI', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
tab

In [ ]:
for i in range(mk):
  if not os.path.exists(path_out+name+str(mk)+'/'+str(i)+'/'):
      os.makedirs(path_out+name+str(mk)+'/'+str(i)+'/')
  clust=np.where(cl_best==i)[0]
  nodes=[ns[j] for j in clust]
  print('\n\n', i, len(clust))
  sub=G.subgraph(nodes)
  if len(clust)>1:
    calculate_graph_features(sub, path_out+name+str(mk)+'/'+str(i)+'/', tab, i)
  else:
    tab.loc[i]=[None for i in range(31)]
    tab.loc[i, 'nn']=1
    tab.loc[i, 0]=nodes[0]
tab.to_csv(path_out+name+'tab.csv')

In [ ]:
tab=pd.read_csv(path_out+name+'tab.csv') #fill names manually
tab

In [ ]:
lay= nx.spring_layout(G_cl)

In [ ]:
# with open(path_out+name+'la.json') as f: #only if outer-made layout is present
#   lay={int(i['id']): np.array((i['x'], i['y'])) for i in json.load(f)['nodes']}

In [ ]:
%matplotlib inline

In [ ]:
for i in G_cl.nodes: #if names not entered manually change 'name' -> '0'
  G_cl.nodes[i]['name']=tab.loc[i]['name']

In [ ]:
plt.figure(figsize=(40, 40), dpi=200)
ec=nx.eigenvector_centrality(G_cl, max_iter=10000)
last=sorted(ec.values())[-max(min(len(G_cl)//2, 70), min(len(G_cl), 70))]
pos=lay
nx.draw_networkx_nodes(G_cl, pos, node_size=[i*50*3.7**(i*15)+20 for i in ec.values()], alpha=0.85,  node_color=[i for i in ec.values()], cmap=plt.cm.Oranges)
nx.draw_networkx_edges(G_cl, pos, alpha=[1 for i in nx.get_edge_attributes(G_cl, 'weight').values()])
for params in labels_list_parameters1(G_cl, pos, ec, 100, last, True):
    nx.draw_networkx_labels(**params)
plt.savefig(path_out+name+'/eig_layout2.jpeg')
plt.show()
plt.close('all')

In [ ]:
nds=np.array(cl_nn)
nds1=np.diagonal(cl_A)

In [ ]:
nds

In [ ]:
plt.figure(figsize=(40, 40), dpi=200)
last=sorted(nds/nds.max())[-max(min(len(G_cl)//2, 70), min(len(G_cl), 70))]
# pos = nx.spring_layout(G_cl)
pos=lay
nx.draw_networkx_nodes(G_cl, pos, node_size=[i*20 for i in nds], alpha=0.85,  node_color=[i for i in nds1], cmap=plt.cm.Oranges)
nx.draw_networkx_edges(G_cl, pos, alpha=[i*10 for i in nx.get_edge_attributes(G_cl, 'weight').values()])
for params in labels_list_parameters1(G_cl, pos,  [max(i, 0.5) for i in nds/nds.max()], 70, last, True):
    nx.draw_networkx_labels(**params)
plt.savefig(path_out+name+'dens_layout2.png')
plt.show()
plt.close('all')

In [ ]:
nx.write_graphml(G_cl, path_out+name+'cluster_graph.graphml')

## Clusters centralities

In [ ]:
tab=pd.DataFrame(columns=[i for i in range(15)])

  ec=nx.eigenvector_centrality(G, max_iter=10000)
  for m, j in enumerate(sorted(ec, key=ec.get, reverse=True)[:15]):
    print(G.nodes()[j]['name'], ' -- ', round(ec[j], 4), end=', ')
    tab.loc[num, m]=G.nodes()[j]['name']
  print()
tab.to_csv(path_out+'evc_cl2.csv')
tab

In [ ]:
tab=pd.DataFrame(columns=[i for i in range(15)])
for num in ['rus/DL/', 'en_r1/DL/', 'en_r123/DL/', 'dutch/DL/', 'USF/DL/', 'rus/BHr+/'
            'rus/NB2/', 'en_r1/NB2/', 'en_r123/NB2/', 'dutch/NB2/', 'USF/NB2/', 'en_r123/NB/', 'dutch/NB/',]:
  G = nx.read_graphml(path_out+num+"cluster_graph.graphml")
  ec=nx.degree_centrality(G)
  for m, j in enumerate(sorted(ec, key=ec.get, reverse=True)[:15]):
    print(G.nodes()[j]['name'], ' -- ', round(ec[j], 4), end=', ')
    tab.loc[num, m]=G.nodes()[j]['name']
  print()
tab.to_csv(path_out+'dc_cl2.csv')
tab

In [ ]:
tab=pd.DataFrame(columns=[i for i in range(15)])
for num in ['rus/DL/', 'en_r1/DL/', 'en_r123/DL/', 'dutch/DL/', 'USF/DL/', 'rus/BHr+/'
            'rus/NB2/', 'en_r1/NB2/', 'en_r123/NB2/', 'dutch/NB2/', 'USF/NB2/', 'en_r123/NB/', 'dutch/NB/',]:
  G = nx.read_graphml(path_out+num+"cluster_graph.graphml")
  ec=nx.in_degree_centrality(G)
  for m, j in enumerate(sorted(ec, key=ec.get, reverse=True)[:15]):
    print(G.nodes()[j]['name'], ' -- ', round(ec[j], 4), end=', ')
    tab.loc[num, m]=G.nodes()[j]['name']
  print()
tab.to_csv(path_out+'idc_cl2.csv')
tab

In [ ]:
tab=pd.DataFrame(columns=[i for i in range(15)])
for num in ['rus/DL/', 'en_r1/DL/', 'en_r123/DL/', 'dutch/DL/', 'USF/DL/', 'rus/BHr+/'
            'rus/NB2/', 'en_r1/NB2/', 'en_r123/NB2/', 'dutch/NB2/', 'USF/NB2/', 'en_r123/NB/', 'dutch/NB/',]:
  G = nx.read_graphml(path_out+num+"cluster_graph.graphml")
  ec=nx.out_degree_centrality(G)
  for m, j in enumerate(sorted(ec, key=ec.get, reverse=True)[:15]):
    print(G.nodes()[j]['name'], ' -- ', round(ec[j], 4), end=', ')
    tab.loc[num, m]=G.nodes()[j]['name']
  print()
tab.to_csv(path_out+'odc_cl2.csv')
tab

In [ ]:
tab=pd.DataFrame(columns=[i for i in range(15)])
for num in ['rus/DL/', 'en_r1/DL/', 'en_r123/DL/', 'dutch/DL/', 'USF/DL/', 'rus/BHr+/'
            'rus/NB2/', 'en_r1/NB2/', 'en_r123/NB2/', 'dutch/NB2/', 'USF/NB2/', 'en_r123/NB/', 'dutch/NB/',]:
  G = nx.read_graphml(path_out+num+"cluster_graph.graphml")
  ec=nx.closeness_centrality(G)
  for m, j in enumerate(sorted(ec, key=ec.get, reverse=True)[:15]):
    print(G.nodes()[j]['name'], ' -- ', round(ec[j], 4), end=', ')
    tab.loc[num, m]=G.nodes()[j]['name']
  print()
tab.to_csv(path_out+'cc_cl2.csv')
tab

In [ ]:
tab=pd.DataFrame(columns=[i for i in range(15)])
for num in ['rus/DL/', 'en_r1/DL/', 'en_r123/DL/', 'dutch/DL/', 'USF/DL/', 'rus/BHr+/'
            'rus/NB2/', 'en_r1/NB2/', 'en_r123/NB2/', 'dutch/NB2/', 'USF/NB2/', 'en_r123/NB/', 'dutch/NB/',]:
  G = nx.read_graphml(path_out+num+"cluster_graph.graphml")
  ec=nx.betweenness_centrality(G)
  for m, j in enumerate(sorted(ec, key=ec.get, reverse=True)[:15]):
    print(G.nodes()[j]['name'], ' -- ', round(ec[j], 4), end=', ')
    tab.loc[num, m]=G.nodes()[j]['name']
  print()
tab.to_csv(path_out+'bc_cl2.csv')
tab

## Homology

In [ ]:
name='USF/SB_4845_0_base0'
G = nx.read_graphml(path_out+name+".graphml")

In [ ]:
A=nx.to_numpy_array(G)
A=A-np.diag(np.diagonal(A))
A=scipy.sparse.csr_matrix(A)
A

In [ ]:
del G

In [ ]:
res1=pyflagser.flagser_unweighted(A)
# res=res1['dgms']
print(name, res1['betti'], res1['cell_count'])

In [ ]:
a='USF/SB_'
b='_base'
last=4845
for i in [last]:
    for j in [0, 1, 2, 3]:
        for k in [0, 100, 200, 500]:
            name=a+str(i)+'_'+str(j)+b+str(k)
            if not os.path.exists(path_out+name+'.graphml'):
                print(name, 'not found')
                continue
            G = nx.read_graphml(path_out+name+".graphml")
            A=nx.to_numpy_array(G)
            A=A-np.diag(np.diagonal(A))
            A=scipy.sparse.csr_matrix(A)
            res1=pyflagser.flagser_unweighted(A)
            print(name, 'betti', res1['betti'], 'cells', res1['cell_count'])

In [ ]:
name='USF/'
G = nx.read_graphml(path_out+name+"USF_main.graphml")

In [ ]:
A=nx.to_numpy_array(G)
A=A-np.diag(np.diagonal(A))
A=scipy.sparse.csr_matrix(A)

In [ ]:
del G

In [ ]:
res2=pyflagser.flagser_weighted(A)['dgms']

In [ ]:
# res2=np.load(path_out+'en_r123/PH/PH.npy', allow_pickle=True)[()]['dgms']

In [ ]:
for i in range(0, min(len(res), len(res2))):
  print(i, len(res[i]), len(res2[i]))
  bn=0
  res[i][res[i]==np.inf]=1
  res2[i][res2[i]==np.inf]=1
  k=0
  for x, y in res[i]:
    k+=1
    if not k% (max(len(res[i])//10, 1)):
      print(k, end=' ')
    best=1
    for x1, y1 in res2[i]:
      dst=max(abs(x-x1), abs(y-y1))
      if dst<bn:
        continue
      if dst<best: 
        best=dst
    if best>bn:
      bn=best
    if bn==1:
      break
  if bn==1:
    continue
  print()
  k=0
  for x, y in res2[i]:
    k+=1
    if not k% (max(len(res2[i])//10, 1)):
      print(k, end=' ')
    best=1
    for x1, y1 in res[i]:
      dst=max(abs(x-x1), abs(y-y1))
      if dst<best: 
        best=dst
    if best>bn:
      bn=best
    if bn==1:
      break
  print(bn)

In [ ]:
# res=np.load(path_out+'en_r123/PH/PH.npy', allow_pickle=True)[()]
# res

In [ ]:
if not os.path.exists(path_out+name+'PH/'):
  os.makedirs(path_out+name+'PH/')

In [ ]:
for dim in range(len(res)-1):
  plt.figure(figsize=(8, 6), dpi=200)
  plt.rcParams['font.size'] = '14'
  x=res['dgms'][dim][:, 0]
  y=res['dgms'][dim][:, 1]
  y[y==np.inf]=1
  print(len(x))
  plt.scatter(x, y, s=5)
  plt.plot([0,1], [0,1],  c='k', alpha=0.2);
  plt.xlabel('birth')
  plt.ylabel('death')
  plt.title('Persistence diagram dim='+str(dim))
  plt.savefig(path_out+name+'PH/PD'+str(dim)+'.jpeg')
  plt.close('All')
  if len(x)<500000:
    plt.figure(figsize=(8, 6), dpi=200)
    pos=[i for i in range(x.shape[0])]
    plt.barh(pos, y-x,  left=x)
    # plt.setxticks(None)
    plt.title('Persistence barcode dim='+str(dim))
    plt.gca().yaxis.set_major_locator(plt.NullLocator())
    plt.xticks([i for i in np.arange(0, 1.1, 0.1)])
    plt.show()
    plt.savefig(path_out+name+'PH/PB'+str(dim)+'.jpeg')
    plt.close('All')
  # plt.legend();

In [ ]:
plt.figure(figsize=(8, 6), dpi=200)
plt.title('Persistence diagram')
for dim in range(len(res['betti'])-1):
  plt.rcParams['font.size'] = '14'
  x=res['dgms'][dim][:, 0]
  y=res['dgms'][dim][:, 1]
  y[y==np.inf]=1
  print(len(x))
  plt.scatter(x, y, s=5, label='dim '+str(dim))
  plt.plot([0,1], [0,1],  c='k', alpha=0.2);
  plt.xlabel('birth')
  plt.ylabel('death')
  # plt.close('All')
  
  # plt.figure(figsize=(8, 6), dpi=200)
  # x=res['dgms'][dim][:, 0]
  # y=res['dgms'][dim][:, 1]
  # y[y==np.inf]=1
  # plt.scatter(x, y, s=5)
  # plt.plot([0,1], [0,1],  c='k', alpha=0.2);
  # plt.title('Persistence diagram dim='+str(dim))
  # plt.close('All')
  plt.legend();
plt.savefig(path_out+name+'PH/PD_all.jpeg')